<font color='red'><h1> Workbook for filter feature engineering </h1></font>

This workbook does feature engineering on row data, it applies feature selectiona and feature extraction. This workbook is to
1. Select features based on filter methods
2. Extract features based on PCA

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.decomposition import PCA

In [2]:
csv_path = os.getcwd() + "/data/"

In [3]:
train_df = pd.read_csv(csv_path + "train.csv")
val_df = pd.read_csv(csv_path + "val.csv")

In [4]:
X_train = train_df.drop('emission',axis=1)
y_train = train_df['emission']
X_val = val_df.drop('emission',axis=1)
y_val = val_df['emission']

## Feature Set 1

In [ ]:
corr_matrix = X_train.corr()

In [ ]:
# Get the column names of the DataFrame
columns = corr_matrix.columns

# Create an empty list to keep track of columns to drop
drop_cols = []

# Loop over the columns
for i in range(len(columns)):
    for j in range(i + 1, len(columns)):
        # Access the cell of the DataFrame
        if corr_matrix.loc[columns[i], columns[j]] > 0.9:
            drop_cols.append(columns[j])

In [ ]:
drop_cols = set(drop_cols)

In [ ]:
X_train_f1 = X_train.drop(columns = drop_cols, axis = 1)
X_val_f1 = X_val.drop(columns = drop_cols, axis = 1)

In [ ]:
X_train_f1['emission'] = y_train
X_val_f1['emission'] = y_val

In [ ]:
X_train_f1.to_csv(csv_path + "train_f1.csv",index=False)
X_val_f1.to_csv(csv_path + "val_f1.csv",index=False)

## Feature Set 2

In [5]:
corr_matrix = X_train.corr()

In [6]:
# Get the column names of the DataFrame
columns = corr_matrix.columns

# Create an empty list to keep track of columns to drop
drop_cols_dict = {}
drop_cols_set = set()
drop_cols_list = []

# Loop over the columns
for i in range(len(columns)):
    
    drop_cols_list = []
    if columns[i] in drop_cols_set:
        continue
        
    for j in range(i + 1, len(columns)):
        
        if columns[j] in drop_cols_set:
            continue
            
        if corr_matrix.loc[columns[i], columns[j]] > 0.8:
            if columns[i] in drop_cols_dict:
                drop_cols_list = drop_cols_dict[columns[i]]
                drop_cols_list.append(columns[j])
                drop_cols_dict[columns[i]] = drop_cols_list
                
            else:
                drop_cols_list.append(columns[i])
                drop_cols_list.append(columns[j])
                drop_cols_dict[columns[i]] = drop_cols_list
            
            drop_cols_set.add(columns[j])

## Apply PCA

In [7]:
for col in drop_cols_dict:
    col_set = drop_cols_dict[col]
    sample_df_train = train_df[col_set]
    sample_df_val = val_df[col_set]
    
    n_components = 1
    
    if len(col_set) > 3:
        n_components = 3
        
    pca = PCA(n_components=n_components)
    X_pca_train = pca.fit_transform(sample_df_train)
    X_pca_val = pca.fit_transform(sample_df_val)
    
    
    for i in range(X_pca_train.shape[1]):
        train_df[col+'_'+str(i)] = X_pca_train[:,i]
        val_df[col+'_'+str(i)] = X_pca_val[:,i]
        
    train_df.drop(columns=col_set,inplace=True)  
    val_df.drop(columns=col_set,inplace=True)

In [13]:
train_df.to_csv(csv_path + "train_f2.csv",index=False)
val_df.to_csv(csv_path+"val_f2.csv",index=False)